In [1]:
!pip install mne

  Using cached mne-1.1.1-py3-none-any.whl (7.5 MB)
  Using cached pooch-1.6.0-py3-none-any.whl (56 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("../data/MNIST-2021/MindBigDataVisualMnist2021-Muse2v0.17.txt", sep=',', header=None) 
# df= pd.DataFrame.to_numpy(User_frame, dtype= np.float64)
# df

In [15]:
df.iloc[:,785:]

,785,786,787,788,789,790,791,792,793,794,...,7434,7435,7436,7437,7438,7439,7440,7441,7442,7443
0,0,0,1.629815e+09,-20.019531,-33.203125,-27.832031,-30.273438,-20.996094,-19.042969,-20.996094,...,-0.315895,-0.319010,-0.322125,-0.325241,-0.328356,-0.331471,-0.334587,-0.337702,-0.340817,-0.343933
1,0,0,1.629815e+09,-24.414062,-23.437500,-32.226562,-28.320312,-21.484375,-30.273438,-32.226562,...,0.037384,-0.003738,-0.044861,-0.085983,-0.127106,-0.168228,-0.209350,-0.250473,-0.291595,-0.332718
2,0,0,1.629815e+09,-34.179688,-30.761719,-18.554688,-20.996094,-16.113281,-21.484375,-37.597656,...,-0.544487,-0.542288,-0.540089,-0.537890,-0.535691,-0.533492,-0.531293,-0.529094,-0.526895,-0.524695
3,0,0,1.629815e+09,-5.859375,-0.976562,-12.695312,-12.695312,-14.160156,-8.300781,-7.324219,...,-0.322822,-0.338215,-0.353609,-0.369002,-0.384395,-0.399789,-0.415182,-0.430576,-0.445969,-0.461363
4,0,0,1.629815e+09,-23.925781,-31.738281,-23.925781,-21.484375,-44.921875,-39.550780,-23.925781,...,-0.566368,-0.548299,-0.530230,-0.512161,-0.494092,-0.476023,-0.457954,-0.439885,-0.421816,-0.403747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,0,0,1.641971e+09,-45.410156,-22.949219,-17.578125,-29.296875,-48.339844,-54.687500,-22.460938,...,0.517768,0.508422,0.499076,0.489730,0.480384,0.471038,0.470599,0.470159,0.469719,0.469279
35996,0,0,1.641971e+09,-22.460938,-16.113281,-36.132812,-45.410156,-28.808594,-14.648438,-14.160156,...,-0.367859,-0.373840,-0.393778,-0.413716,-0.433654,-0.453593,-0.473531,-0.493469,-0.513407,-0.533345
35997,0,0,1.641971e+09,-14.648438,-45.410156,-58.593750,-34.667970,-1.464844,3.417969,-36.621094,...,-0.305481,-0.435791,-0.566101,-0.696411,-0.826721,-0.957030,-0.938712,-0.920394,-0.902076,-0.883758
35998,0,0,1.641972e+09,-39.550780,-20.996094,-8.789062,-11.718750,-35.156250,-47.851562,-24.902344,...,-0.011215,-0.014954,-0.018692,-0.022430,-0.026169,-0.029907,-0.033646,-0.037384,-0.041122,-0.044861


In [5]:
df.tail(20)

,0,1,2,3,4,5,6,7,8,9,...,7434,7435,7436,7437,7438,7439,7440,7441,7442,7443
35980,TRAIN,-1,-1,0,0,0,0,0,0,0,...,-0.326175,-0.325241,-0.324306,-0.323372,-0.322437,-0.321502,-0.322904,-0.324306,-0.325708,-0.327110
35981,TRAIN,17990,8,0,0,0,0,0,0,0,...,-0.332568,-0.324792,-0.317016,-0.309240,-0.301465,-0.293689,-0.285913,-0.278137,-0.270361,-0.262585
35982,TRAIN,-1,-1,0,0,0,0,0,0,0,...,0.029907,0.059814,0.089722,0.119629,0.149536,0.052338,0.000000,0.011503,0.023006,0.034508
35983,TRAIN,17991,4,0,0,0,0,0,0,0,...,-0.239258,0.201874,0.161499,0.121124,0.080749,0.040375,0.000000,-0.040375,-0.080749,-0.121124
35984,TRAIN,-1,-1,0,0,0,0,0,0,0,...,0.061951,0.097732,0.133514,0.169296,0.205078,0.240860,0.276642,0.254211,0.231781,0.209350
35985,TRAIN,17992,2,0,0,0,0,0,0,0,...,-0.149536,-0.112152,-0.074768,-0.037384,-0.095703,-0.154022,-0.212341,-0.270660,-0.328979,-0.387298
35986,TRAIN,-1,-1,0,0,0,0,0,0,0,...,-0.454216,-0.452346,-0.450477,-0.448608,-0.456085,-0.463562,-0.471038,-0.478515,-0.485992,-0.493469
35987,TRAIN,17993,3,0,0,0,0,0,0,0,...,0.073771,0.092214,0.110657,0.129099,0.147542,0.165985,0.184428,0.202871,0.221313,0.239756
35988,TRAIN,-1,-1,0,0,0,0,0,0,0,...,-0.031227,0.030787,0.092800,0.154814,0.216827,0.170647,0.124467,0.078286,0.032106,-0.014074
35989,TRAIN,17994,9,0,0,0,0,0,0,0,...,0.119629,0.074768,0.029907,-0.014954,-0.059814,-0.104675,-0.149536,-0.155144,-0.160751,-0.166359


In [16]:
df = pd.read_csv("../data/MNIST-2014/MNIST-2014.txt", sep=',', header=None) 
df

FileNotFoundError: [Errno 2] No such file or directory: '../data/MNIST-2014/MNIST-2014.txt'